In [ ]:
## ------------------------------ ##
## SUBSET GEOSPATIAL DATA TO BPBC ##
## ------------------------------ ##

shp_file = gpd.read_file('subset_test_shp/BPBC_Divisions/BPBC_Divisions-IDTM.shp') #open shapefile
names = shp_file['DivisionNa']
files = glob.glob('lcmap_files/*.tiff') #get all the years of cdl imagery
data =[]
for i in range(len(files)):
  data.append(rso.open(files[i])) #open cdl image and append to a list
shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
years = np.arange(1987, 2021) #years of LCMAP data
collection = []
for i in range(len(shp)):
  for n in range(len(years)):
    dataset = data[n]
    year_out = dataset.name[21:-15]
    extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
    coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
    out_img, out_transform = mask(dataset=data[n], shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
    out_meta = data[n].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform})
    # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
    in_file = files[n]
    out_filename = os.path.join('lcmap_files/lcmap_masked_bpbc/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
    out_tiff = rso.open(out_filename, 'w', **out_meta)
    out_tiff.write(np.squeeze(out_img, 0), 1)
    out_tiff.close()
    collection.append(out_img)

In [ ]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##
years = np.arange(1987,2021)
temporal_group = []
for i in range(len(names)):
  files= sorted(glob.glob('lcmap_files/lcmap_masked_bpbc/'+names[i]+'_*.tif')) #name for all the csv files
  sta = SpatioTemporalAnalysis(files, dates=years, nodata=0) #import all CDL rasters and mask
  temporal_group.append(sta)

In [ ]:
# ------------------------------------ #
# CALCULATE THE CLASS PROPORTIONS BPBC #
# ------------------------------------ #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('lcmap_files/proportions/bpbc_metrics/long/'+names[i]+'_prop.csv')
  proportions.append(df)

In [ ]:
# ------------------------------------ #
# CALCULATE CONFIGURATION METRICS BPBC #
# ------------------------------------ #

config = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['contagion', 'largest_patch_index'])
  df.to_csv('lcmap_files/proportions/bpbc_metrics/long/'+names[i]+'_configuration.csv')
  config.append(df)

In [ ]:
# ---------------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics BPBC #
# ---------------------------------------------------------------------- #

# Import csv files into a list of dataframes

files = sorted(glob.glob('lcmap_files/proportions/bpbc_metrics/long/*_prop.csv'))
files_config = sorted(glob.glob('lcmap_files/proportions/bpbc_metrics/long/*_configuration.csv'))
names = list(sorted(shp_file['DivisionNa']))

proportions = []
for i in files:
  data = pd.read_csv(i)
  proportions.append(data)
config = []
for i in files_config:
  data = pd.read_csv(i)
  config.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DivName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)


In [ ]:
## ----------------------------------------------- ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING BPBC ##
## ----------------------------------------------- ##

prop = pd.concat(new_df)

change = prop.groupby('DivName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('lcmap_files/proportions/bpbc_change.csv')